In [ ]:
from google.colab import files

uploaded = files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!pip install kaggle
!kaggle competitions download -c thaijo-researcher

  0% 0.00/290k [00:00<?, ?B/s]
100% 290k/290k [00:00<00:00, 99.8MB/s]


In [ ]:
!unzip "/content/thaijo-researcher.zip" -d "/content/datasets"

Archive:  /content/thaijo-researcher.zip
  inflating: /content/datasets/sample_submission.csv  
  inflating: /content/datasets/test.csv  
  inflating: /content/datasets/thaijo_cleansing.py  


In [ ]:
import pandas as pd
import regex as re
import numpy as np
import unicodedata
from datetime import datetime

ROOT = "/content/datasets/"

def remove_spaces(s:str):
    return s.strip()

df = pd.read_csv(ROOT + 'test.csv')
df = df.fillna('')
df = df[["_id", "_source.author", "_source.co-author"]]
columns = ["id", "author", "coauthor"]
df.columns = columns

In [ ]:
df["author_contain_colon"] = df["author"].apply(
    lambda x: 1 if x.find(",") != -1 else 0
)
df[df["author_contain_colon"] == 1]

,id,author,coauthor,author_contain_colon
2445,82331,"Rasda Boonprasert,","Rasda Boonprasert,, Paiboon Tummarintra, Pinit...",1


In [ ]:
df["author"] = df["author"].apply(lambda x: x.strip(",").replace(";", ","))
df[df["author_contain_colon"] == 1]

,id,author,coauthor,author_contain_colon
2445,82331,Rasda Boonprasert,"Rasda Boonprasert,, Paiboon Tummarintra, Pinit...",1


In [ ]:
from typing import Callable

def is_starts_with_word(string:str, word:str):
    pattern = r'^\b' + re.escape(word)
    return bool(re.search(pattern, string))

def is_contain_word(string:str, word:str):
    pattern = re.escape(word)
    return bool(re.search(pattern, string))

def check_first_coauthor(author: str, coauthors: str):
    return 1 if coauthors[: len(author)] == author else 0


df["same_first_coauthor"] = df.apply(
    lambda x: check_first_coauthor(x.author, x.coauthor), axis=1
)
df["same_first_coauthor"].value_counts()

1    4703
0      27
Name: same_first_coauthor, dtype: int64

In [ ]:
def reg_split(inp,mark=';'):
    ls = []
    start = 0
    rrrr = 0
    while rrrr!=None:
        prev = ''
        rrrr = re.search(mark,inp)
        if rrrr == None:
            break
        start = rrrr.start()+1
        if (len(re.findall('\(',inp[: rrrr.start()])) == len(re.findall('\)',inp[: rrrr.start()]))) or ('(' not in inp[: rrrr.start()]):
            ls.append(inp[: rrrr.start()])
        else:
            prev = inp[:rrrr.start()+1]
            inp = inp[start:]
            rrrr = re.search(mark,inp)
            if rrrr == None:
                ls.append(prev+inp)
            else:
                ls.append(prev+inp[: rrrr.start()])
            start = rrrr.start()+1
        inp = inp[start:]
    ls.append(inp)
    if len(ls) == 1 and '(' not in ls[0] and ')' not in ls[0]:
        tmp = re.sub('[ ]+',' ',ls[0])
        if len(tmp.split()) > 4 and len(tmp.split())%2 == 0:
            nls = tmp.split()
            ans = []
            prev = ''
            for var in range(len(nls)):
                if var%2 == 1:
                    ans.append(prev+' '+nls[var])
                else:
                    prev = nls[var]
            ls = ans
    return ls

In [ ]:
def remove_restrict(col: str):
    res_words = [
        'Et Al','', '', '',
        "Editorial team",
        "บทบรรณาธิการ (Editorial)",
        'กองบรรณาธิการ ผู้เชี่ยวชาญ',
        'กองบรรณาธิการ',
        "",
        "ผู้ทรงคุณวุฒิ -",
        "แนะนำผู้เขียน -",
        "ศูนย์บริการโลหิตแห่งชาติ สภากาชาดไทย",
        "คำแนะนำสำหรับผู้เขียน วารสารวิทยาการจัดการ",
        "กสทช.",
        "และคณะ",
        "and Other",
        "บทบรรณาธิการ",
        'บรรณาธิการ',
        "-- --",
        "- -",

        ". .",
        " -",
        "Authors :",
        "(ผู้แต่ง)",
        "*",
        " ๋",
        "dusayu@kku.ac.th",
        'et.al',
        'CFA',
        'M.D.',
        'Asst.',
    ]
    non_conflict_nametitles = [
        r'พันเอก',
        r"ร้อยเอก",
        r"ม.ร.ว.",
        r"รองศาสตราจารย์",
        r"รศ.",
        r"ผู้ช่วยศาสตราจารย์",
        r"ผศ.",
        r"ศาสตราจารย์",
        r"อ.ดร.",
        r"ดร.",
        r"อาจารย์",
        r"เด็กหญิง",
        r"เด็กชาย",
        r"ว่าที่ร้อยตรี",
        r"ว่าที่ ร.ต.",
        r"ร.ต.อ.",
        r"ว่าที่ พ.ต.",
        r"นพ.",
        r"Mr.",
        r"Ms.",
        r"Jr.",
        r"Dr.",
        r"Ph.D",
        r"Assoc.",
        r"Assist.",
        r"Prof.",
        r"Lect.",
        r"Ven.",
        r", M.R.",
        r'Mohd.',
        r"ศ.",
        r' .',
    ]
    col = col.lower()
    col = re.sub(r"[0-9]+", "", col)
    col = col.replace("(phrakrusuteejariyawat)", ", phrakrusuteejariyawat")
    col = col.replace("somboon ๋jarukasemthawee", "somboon jarukasemthawee")
    col = col.replace("จตุพล รั กเปี่ยม", "จตุพล รักเปี่ยม")
    for res_word in res_words + non_conflict_nametitles:
        col = col.replace(res_word.lower(), "")
    return col.title()

def seperate_names(col: str):
    col = col.strip().strip(",").strip()
    names = col.split(",")
    if col == names[0] and len(names[0]) > 50:
        names = reg_split(names[0],mark=';')
    return [name.strip() for name in names if name.strip() != ""]

def filter_with_general_rules(name: str):
    start_with_terms = [
        "กองบรรณาธิการ",
        "คณะ",
        "สำนัก",
        "Cover ",
        "เจ้าหน้าที่",
    ]
    contain_terms = [
        "Editorial"
        "Author",
        "Journal",
        "วารสาร",
        "มหาวิทยาลัย",
        "วิทยาลัย",
        "Thailand",
        "Subcommittee"
    ]
    for term in start_with_terms:
        if is_starts_with_word(name, term):
            return False
    for term in contain_terms:
        if is_contain_word(name, term):
            return False
    return True

In [ ]:
def handle_blanket_phra_case(name: str):
    phra_titles = [
        'พระเทพปริยัติมุนี',
        "พระเทพ",
        'พระอธิการ',
        "พระปลัด",
        "พระครูใบฎีกา",
        "พระครูปลัด",
        'พระครูสมุห์',
        "พระครู",
        "พระมหา",
        "พระ",
        "Phrakrubha",
        "Phrakhru",
        "Phrakru",
        "Phramaha",
        "Phra"
    ]

    for title in phra_titles:
        name = name.replace(title, "")
    name = name.replace("( ", " (").replace("(", " (")
    name = name.replace(" )", ") ").replace(")", ") ")
    name = name.replace("  ", " ").strip()

    return name


def handle_blanket_general_case_1(name: str):
    splited_names = name.replace("(", ";").replace(")", ";").split(";")
    splited_names = [name.strip() for name in splited_names if name.strip() != ""]
    if len(splited_names) % 2 == 0:
        lan1_name = ""
        lan2_name = ""
        for i in range(len(splited_names) // 2):
            lan1_name += splited_names[i * 2] + " "
            lan2_name += splited_names[(i * 2) + 1] + " "
        name = lan1_name.strip() + "," + lan2_name.strip()
    else:
        name = " ".join(splited_names)
    return name

def handle_blanket_general_case_2(name: str):
    splited_names = name.replace("[", ";").replace("]", ";").split(";")
    splited_names = [name.strip() for name in splited_names if name.strip() != ""]
    if len(splited_names) % 2 == 0:
        lan1_name = ""
        lan2_name = ""
        for i in range(len(splited_names) // 2):
            lan1_name += splited_names[i * 2] + " "
            lan2_name += splited_names[(i * 2) + 1] + " "
        name = lan1_name.strip() + "," + lan2_name.strip()
    else:
        name = " ".join(splited_names)
    return name


def handle_blanket(name: str):
    if is_starts_with_word(name, "พระ") or is_starts_with_word(name, "Phra"):
        name = handle_blanket_phra_case(name)
    elif is_contain_word(name, "("):
        name = handle_blanket_general_case_1(name)
    elif is_contain_word(name, "["):
        name = handle_blanket_general_case_2(name)
    return name

def handle_th_and(name:str):
    and_index = name.find(" และ")
    if and_index != -1:
        next_space_index = name[and_index+1:].find(" ")
        if next_space_index != -1:
            name = ",".join(name.split(" และ"))
    return name

def handle_th_en_name(name: str):
    break_index = None
    try:
        for i in range(1, len(name) - 1):
            prev_lang = unicodedata.name(name[i - 1]).split(" ")[0]
            break_lang = unicodedata.name(name[i]).split(" ")[0]
            next_lang = unicodedata.name(name[i + 1]).split(" ")[0]
            if break_lang == "SPACE" and set([prev_lang, next_lang]) == {
                "THAI",
                "LATIN",
            }:
                break_index = i
                break
    except:
        pass

    if break_index is not None:
        first_lang_name = name[:break_index]
        second_lang_name = name[break_index + 1 :]
        name = first_lang_name + "," + second_lang_name
    return name

def captitalize_first_letter(name:str):
    name_parts = name.split(" ")
    name_parts_capitalized = list()
    for name_part in name_parts:
        name_part = re.sub(r'', '์', name_part)
        if re.search(r'[ก-ฮ]', name_part):
            name_part = re.sub(r'[a-zA-Z]', '', name_part)
            if name_part[:3] == "นาย" or name_part[:3] == "นาง":
                name_part = re.sub(r'นาย', '', name_part)
                name_part = re.sub(r'นางสาว', '', name_part)
                name_part = re.sub(r'นาง', '', name_part)
        elif re.search(r'[a-zA-Z]', name_part):
            name_part = re.sub(r'[์ิึืุู]', '', name_part)
            name_part = re.sub(r'[์ิึืุู]', '', name_part)
            name_part = name_part.title()
        if len(name_part) >= 2 and name_part[1] == ".":
            name_parts_capitalized.append(name_part)
        else:
            if "(" in name_part or ")" in name_part:
                name_parts_capitalized.append(name_part)
            else:
                name_parts_capitalized.append(name_part.capitalize())
        name_part = name_part.replace("วรวรรณ ตุมมงคล", "วรวรรณ ตุ มมงคล")
        name_part = name_part.replace("เสาวลักษณ์ กูเจริญประสิทธิ์", "เสาวลักษณ์ กู เจริญประสิทธิ์")
    return " ".join(name_parts_capitalized)

s = "A. A.E. A.e. Word-With-hyphen"
captitalize_first_letter(s)

'A. A.E. A.E. Word-with-hyphen'

In [ ]:
def get_names(col: str):
    col = remove_restrict(col)
    #print(f"layer 0: {col}")
    names = seperate_names(col)
    #print(f"layer 1: {names}")
    names = [name.replace("  ", " ").strip("(") for name in names]
    #print(f"layer 2: {names}")
    names = [name for name in names if filter_with_general_rules(name)]
    #print(f"layer 3: {names}")
    names = [handle_blanket(name) for name in names]
    #print(f"layer 4: {names}")
    names = ",".join(names).split(",")
    #print(f"layer 5: {names}")
    names = [handle_th_and(name) for name in names]
    #print(f"layer 6: {names}")
    names = ",".join(names).split(",")
    #print(f"layer 7: {names}")
    names = [handle_th_en_name(name) for name in names]
    #print(f"layer 8: {names}")
    names = ",".join(names).split(",")
    #print(f"layer 9: {names}")
    names = [captitalize_first_letter(name) for name in names]
    #print(f"layer 10: {names}")
    return names

test_col = "ศศิธร ชิดนายี"
get_names(test_col)

['ศศิธร ชิดนายี']

In [ ]:
get_names("จตุพล รั กเปี่ยม")

['จตุพล รักเปี่ยม']

In [ ]:
get_names('มณฑล ทิพย์ศรี, วรวรรณ ตุมมงคล, เสาวลักษณ์ กูเจริญประสิทธิ์')

['มณฑล ทิพย์ศรี', 'วรวรรณ ตุมมงคล', 'เสาวลักษณ์ กูเจริญประสิทธิ์']

In [ ]:
get_names('สุขวสา ภวภูตานนท์ ณ มหาสารคาม, สุทธิพงศ์ หกสุวรรณ')

['สุขวสา ภวภูตานนท์ ณ มหาสารคาม', 'สุทธิพงศ์ หกสุวรรณ']

In [ ]:
get_names('พระครูปลัดบำรุง พระครูปลัดบำรุง  ปญฺญาพโล โพธิ์ศรี, พระเทพปริยัติมุนี วีรปญฺโญ')

['บำรุง บำรุง ปญฺญาพโล โพธิ์ศรี', 'วีรปญฺโญ']

In [ ]:
def export_submission(df: pd.DataFrame):
    submission = {"id": [], "name": []}
    for row in df.itertuples():
        submission["id"].extend([f"{row.id}_{i+1}" for i in range(10)])
        names = list()
        author: str = row.author
        if author != "":
            names.extend(get_names(author))
        coauthor: str = row.coauthor
        if coauthor != "":
            names.extend(get_names(coauthor))
        if len(names) == 0:
            submission["name"].extend([""] * 10)
            continue
        unique_indexes = np.unique(names, return_index=True)[1]
        names = [names[index] for index in sorted(unique_indexes)]
        if len(names) > 10:
            names = names[:10]
        elif len(names) < 10:
            names.extend([""] * (10 - len(names)))
        submission["name"].extend(names)
    submission_df = pd.DataFrame(submission)
    save_path = f"submission_final_final_final_final.csv"
    submission_df.to_csv(save_path, index=False)

In [ ]:
export_submission(df)

In [ ]:
pd.set_option('display.max_rows', None)

df_old = pd.read_csv('/content/submission_final_final_final.csv')
df_new = pd.read_csv('/content/submission_final_final_final_final.csv')

diff = df_old.compare(df_new)
diff = pd.merge(diff, df_old[['id']], left_index=True, right_index=True, suffixes=('_old', '_new'))
diff

<ipython-input-900-ca8e869e0ac5>:7: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  diff = pd.merge(diff, df_old[['id']], left_index=True, right_index=True, suffixes=('_old', '_new'))


,"(name, self)","(name, other)",id
9970,จตุพล รั กเปี่ยม,จตุพล รักเปี่ยม,161044_1
27280,บรรณาธิการ,NaN,16581_1
37011,Et Al,NaN,83313_2


In [ ]:
len(diff)

3

In [ ]:
pd.set_option('display.max_rows', None)

df_old = pd.read_csv('/content/predict_ver13.csv')
df_new = pd.read_csv('/content/submission_final_final_final_final.csv')

diff = df_old.compare(df_new)
diff = pd.merge(diff, df_old[['id']], left_index=True, right_index=True, suffixes=('_old', '_new'))
diff

<ipython-input-902-b1e0a6223908>:7: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  diff = pd.merge(diff, df_old[['id']], left_index=True, right_index=True, suffixes=('_old', '_new'))


,"(name, self)","(name, other)",id
1390,Sitthimongkhon Sanor,Tepsitthimongkhon Sanor,116315_1
2292,สุธีจริยวัฒน์ (Phrakrusuteejariyawat),สุธีจริยวัฒน์,58249_3
2293,รัฐสภา พงษ์ภิญโญ,Suteejariyawat,58249_4
2294,Rattasapa Pongpinyo,รัฐสภา พงษ์ภิญโญ,58249_5
2295,NaN,Rattasapa Pongpinyo,58249_6
9970,จตุพล รั กเปี่ยม,จตุพล รักเปี่ยม,161044_1
12721,Somboonjarukasemthawee,Somboon Jarukasemthawee,102418_2
13240,Somchai Payago,Palad Somchai Payago,165010_1
13691,M.HUM M.HUM,M.Hum M.Hum,157973_2
14590,Sittichai Phrasittichai,Sittichai Sittichai,148929_1


In [ ]:
len(diff)

24